In [1]:
pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 4.7 MB/s eta 0:00:00


In [2]:
pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.1 MB/s eta 0:00:00


In [3]:
pip install python-dotenv


In [4]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = ['']

In [7]:
pip install pypdf

In [8]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("short-stories-for-children.pdf")
pages = loader.load()
pages = pages[4:]

In [9]:
len(pages)

40

In [10]:
page = pages[0]
print(page.page_content[0:500])

 
 
Spoken English: Short Stories  
5 
 LEVEL 1 : STORI ES FOR PRIMARY SCHOOL CHILDREN  
THE WIND AND THE SUN  
 
Once the Wind and the Sun had an argument. “I am stronger than you,” said the Wind. “No, 
you are not,” said the Sun. Just at that moment they saw a traveler walking across the road. 
He was wrapped in a s hawl. The Sun and the Wind agreed that whoever could separate the 
traveller from his shawl was stronger.  
The Wind took the first turn. He blew with all his might to tear the tra


In [11]:
page.metadata

{'source': 'short-stories-for-children.pdf', 'page': 4}

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

In [13]:
chunk_size = 1200
chunk_overlap = 100

In [14]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
        separators=[" "]
)

In [15]:
stories = r_splitter.split_documents(pages)
# stories

In [16]:
markdown_document = stories

In [17]:
from langchain.text_splitter import MarkdownHeaderTextSplitter

In [18]:
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

In [19]:
txt = ' '.join([d.page_content for d in markdown_document])

In [20]:
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on
)
md_header_splits = markdown_splitter.split_text(txt)
# md_header_splits[0]

In [21]:
from langchain.text_splitter import TokenTextSplitter

In [22]:
pip install nltk

In [23]:
from langchain.text_splitter import NLTKTextSplitter
import nltk
nltk.download('punkt')

text_splitter = NLTKTextSplitter(chunk_size=1300)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [24]:
texts = text_splitter.split_text(txt)
# texts[0]


In [25]:
len(texts)

81

In [26]:
 pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 966.7/966.7 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 99.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 75.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 7.9 MB/s 

In [27]:
from langchain.vectorstores import Chroma

In [28]:
persist_directory = 'docs/chroma/'

In [29]:
!rm -rf ./docs/chroma  # remove old database files if any

In [30]:
with open('document.txt', 'w') as file:
    for item in texts:
        file.write(item + '\n')

In [31]:
# import
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader

In [32]:
loader = TextLoader("document.txt")
documents = loader.load()

In [33]:
text_splitter = CharacterTextSplitter(chunk_size=1200, chunk_overlap=50)
docs = text_splitter.split_documents(documents)


In [36]:
!pip install sentence_transformers
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.6 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=8cc9acb3667d6b08f6a36d1d50b53523e1be46a9e5ca4acb38c3f538c9f7a64d
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [37]:
db = Chroma.from_documents(docs, embedding_function)


In [38]:
query = "hardworking man's story"
docs = db.similarity_search(query, k=3)

In [39]:
print(docs[1].page_content)



The father smiles a nd says: "My son, pay attention."

Again the man screams: "You are a champion!"

The voice answers: "You are a champion!"

The boy is surprised, but does not understand.

Then the father explains: "People call this ECHO, but really this is LIFE.

It gives you b ack everything you say or do.

Our life is simply a reflection of our actions.

If you want more love in the world, create more love in your heart.

If you want more competence in your team, improve your competence.
Our life is simply a reflection of our actions.

If you want more love in the world, create more love in your heart.

If you want more competence in your team, improve your competence.

This relationship applies to everything, in  all aspects of life;  
Life will give you back everything you have given to it."

YOUR LIFE IS NOT A COINCIDENCE.

IT'S A REFLECTION OF YOU!"

Spoken English: Short Stories  
16 
 THE MISER  
A miser sold all that he had and bought a lump of gold, which he buried in a ho

In [40]:
# save to disk
db2 = Chroma.from_documents(docs, embedding_function, persist_directory="./chroma_db")
db2.persist()
docs = db.similarity_search(query)

# load from disk
db3 = Chroma(persist_directory="./chroma_db")
docs = db.similarity_search(query)
print(docs[0].page_content)

The grasshopper 
pulled a long face and walked away.

So we say "Work is real worship".

NEVER BE UNGRATEFUL  
 
It was high summer.

The sun was extremely hot.

Two travelers were going along a dusty 
road that had no trees along its sides.
So we say "Work is real worship".

NEVER BE UNGRATEFUL  
 
It was high summer.

The sun was extremely hot.

Two travelers were going along a dusty 
road that had no trees along its sides.

Looking for some shelter from the hot sun, they saw a 
tree with big leaves and branches spread like an umbrella.

shelter from the hot sun, they saw a 
tree with big leaves and branches spread like an umbrella.

They placed their b elongings on the ground and sat in the cool thick shade of the tree.

After 
taking some rest, one traveler said to the other, "What a useless tree it is!

It bears no fruits 
at all."

Hearing this, the tree felt pinched and burst out, "You ungrateful soul!

O n one hand, you are 
taking shelter in my cool shade from the burning heat

In [41]:
docs_mmr = db.max_marginal_relevance_search(query,k=3)

In [42]:
docs_mmr[1].page_content

'"There are 5 things you need to know," he told the pencil, "Before I send you out into the \nworld.\n"There are 5 things you need to know," he told the pencil, "Before I send you out into the \nworld.\n\nAlways remember them and never forget, and you will become the be st pencil you \ncan be."\n\n"One: You will be able to do many great things, but only if you allow yourself to be held in \nSomeone\'s hand."\n\n"Two: You will experience a painful sharpening from time to time, but you\'ll need it to \nbecome a better pencil."\n\n"Three: Yo u will be able to correct any mistakes you might make."\n\n"Four: The most important part of you will always be what\'s inside."\n\n"And Five: On every surface you are used on, you must leave your mark.\n\nNo matter what the \ncondition, you must continue to write."\n\nThe pencil understood and promised to remember, and went into the box with purpose in \nits heart.\n\nNow replacing the place of the pencil with you.\n\nAlways remember them and never f

In [43]:
db._collection.count()

89

In [73]:
from langchain.chat_models import ChatOpenAI
llm_name = "gpt-3.5-turbo-0301"
llm = ChatOpenAI(openai_api_key = '', model_name=llm_name, temperature=0.5)

In [74]:
# retrieval qa
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=db.as_retriever()
)

In [82]:
result = qa_chain({"query": "grasshoper"})

In [83]:
result["result"]

'What is the story of the grasshopper? \n\nOnce upon a time, a grasshopper was basking in the warm sun on a fine sunny day in winter. He was very hungry as he had not eaten anything since last night. So he looked around to find something to soothe his hunger. Suddenly, he saw a few ants carrying grains into their hole. He went up to the ants and asked humbly if they could spare a few grains for him as he hadn\'t eaten anything since yesterday and was almost starving to death. One of the ants asked the grasshopper what he was doing the whole summer and why he didn\'t store up food for the winter season. The grasshopper replied that he spent all summer singing songs and couldn\'t store anything. The ant chuckled and remarked, "Then dance the winter away." So the moral of the story is "Work is real worship" and "Never be ungrateful."'

In [ ]:
# adding memory